In [119]:
# match data with aggregated individual data
import pandas as pd
match_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/player_data/data/matches_with_player_agg.csv'
matches = pd.read_csv(match_path)

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Features win correlation

Looking for which features are the most predictive in determining the winner

In [20]:
# Find the date for which each feature came available
available_date = {}
for col in player_cols_to_agg:
    h_col = 'h_' + col
    first = matches[(matches[h_col] != 0) & (matches[h_col].isna() == False)]['date'].iloc[0]
    first_season = int(first[0:4])
    available_date[col] = first_season

In [21]:
# Find win correlations for each attribute
corr = win_corr(ladder_columns + player_cols_to_agg + misc_columns)

In [23]:
corr.sort(key = lambda x: x[1], reverse=True)
corr[0:20]

[('wins', 0.6753904601097509),
 ('prem_points', 0.6731864322852191),
 ('percentage', 0.6656091370558376),
 ('percentage_form', 0.64859975881013),
 ('season_Supercoach', 0.6452282157676349),
 ('points_for', 0.6312553011026294),
 ('season_score_involvements', 0.619979402677652),
 ('season_goal_assists', 0.6145569620253165),
 ('career_brownlow', 0.6138152610441767),
 ('career_games_played', 0.6131127779266006),
 ('season_AFLfantasy', 0.6130705394190872),
 ('season_goals', 0.6063487811411733),
 ('career_goal_assists', 0.604746835443038),
 ('career_goals', 0.59583444950442),
 ('season_inside50', 0.593069306930693),
 ('season_effective_disposals', 0.5892116182572614),
 ('season_marks_in_50', 0.5874741200828157),
 ('season_metres_gained', 0.5859938208032955),
 ('season_brownlow', 0.5857028112449799),
 ('career_kicks', 0.5828412643278916)]

In [29]:
# metric_dict allows us to go through with a finer comb by looking at how predictive each attribute is for each season
metric_dict['wins']

defaultdict(<function __main__.<lambda>()>,
            {1897: 0.7857142857142857,
             1898: 0.7719298245614035,
             1899: 0.7307692307692307,
             1900: 0.6415094339622641,
             1901: 0.7358490566037735,
             1902: 0.7884615384615384,
             1903: 0.7758620689655172,
             1904: 0.6470588235294118,
             1905: 0.6896551724137931,
             1906: 0.7636363636363637,
             1907: 0.49019607843137253,
             1908: 0.647887323943662,
             1909: 0.6857142857142857,
             1910: 0.7671232876712328,
             1911: 0.7183098591549296,
             1912: 0.7285714285714285,
             1913: 0.8356164383561644,
             1914: 0.6911764705882353,
             1915: 0.7678571428571429,
             1916: 0.7,
             1917: 0.5714285714285714,
             1918: 0.7111111111111111,
             1919: 0.7413793103448276,
             1920: 0.76,
             1921: 0.631578947368421,
           

We can also check how often the odds predict the winner

In [67]:
odds_corr

0.7107868681605003

In [68]:
odds_dict

defaultdict(<function __main__.<lambda>()>,
            {2010: 0.625,
             2011: 0.7593582887700535,
             2012: 0.7676767676767676,
             2013: 0.7424242424242424,
             2014: 0.696969696969697,
             2015: 0.7106598984771574,
             2016: 0.7272727272727273,
             2017: 0.6666666666666666,
             2018: 0.6888888888888889,
             2019: 0.6818181818181818})

# Metadata and functions

In [6]:
player_cols_to_agg = [
    'AFLfantasy',
    'Supercoach',
    'behinds',
    'bounces',
    'brownlow',
    'clangers',
    'clearances',
    'contested_marks',
    'contested_poss',
    'disposals',
    'frees_against',
    'frees_for',
    'goal_assists',
    'goals',
    'handballs',
    'hitouts',
    'inside50',
    'kicks',
    'marks',
    'marks_in_50',
    'one_percenters',
    'rebound50',
    'tackles',
    'tog',
    'uncontested_poss',
    'centre_clearances',
    'disposal_efficiency',
    'effective_disposals',
    'intercepts',
    'metres_gained',
    'stoppage_clearances',
    'score_involvements',
    'tackles_in_50',
    'turnovers'
]

match_cols = [
    'odds',
    'line'
]

ladder_columns = [
    'wins',
    'losses',
    'draws',
    'prem_points',
    'played',
    'points_for',
    'points_against',
    'percentage',
    'position'
]

misc_columns = [
    'games_played'
]

In [12]:
# function to tell us what the biggest predictors of a victory are. Essentially the test will be to see
# how often the team that wins each category turns out to be the winner
def win_corr(cols):
    
    corr = []
    
    for col in cols:
        if col in ladder_columns: # Ladder column conditions
            h_col = 'h_' + col
            a_col = 'a_' + col
            h_col_form = 'h_' + col + '_form'
            a_col_form = 'a_' + col + '_form'
            #first_season = available_date[col]
            first_season = 1897

            relevant_games = matches[(matches['season'] >= first_season) &
                                    (matches[a_col] != matches[h_col])] # Added this to remove games where they are same
            relevant_games_form = matches[(matches['season'] >= first_season) &
                                    (matches[a_col_form] != matches[h_col_form])]
            
            total_games = len(relevant_games)
            total_games_form = len(relevant_games_form)
            col_corr = (sum((matches[h_col] > matches[a_col]) & (matches['winner'] == 'home')) 
                          + sum((matches[h_col] < matches[a_col]) & (matches['winner'] == 'away'))) \
                          / total_games

            col_corr_form = (sum((matches[h_col_form] > matches[a_col_form]) & (matches['winner'] == 'home'))
                          + sum((matches[h_col_form] < matches[a_col_form]) & (matches['winner'] == 'away'))) \
                          / total_games_form

            corr.append((col, col_corr))
            corr.append((col + '_form', col_corr_form))
            
        if col in player_cols_to_agg: # Aggregated player stats conditions
            h_col_career = 'h_career_' + col
            a_col_career = 'a_career_' + col
            h_col_season = 'h_season_' + col
            a_col_season = 'a_season_' + col
            first_season = available_date[col]

            relevant_games = matches[matches['season'] >= first_season]
            total_games = len(relevant_games)
            col_corr_career = (sum((matches[h_col_career] > matches[a_col_career]) & (matches['winner'] == 'home')) 
                          + sum((matches[h_col_career] < matches[a_col_career]) & (matches['winner'] == 'away'))) \
                          / total_games

            col_corr_season = (sum((matches[h_col_season] > matches[a_col_season]) & (matches['winner'] == 'home'))
                          + sum((matches[h_col_season] < matches[a_col_season]) & (matches['winner'] == 'away'))) \
                          / total_games

            corr.append(('career_' + col, col_corr_career))
            corr.append(('season_' + col, col_corr_season))
            
        if col in misc_columns: # For games played
            h_col_career = 'h_career_' + col
            a_col_career = 'a_career_' + col
            h_col_season = 'h_season_' + col
            a_col_season = 'a_season_' + col
            first_season = 1897

            relevant_games = matches[matches['season'] >= first_season]
            total_games = len(relevant_games)
            col_corr_career = (sum((matches[h_col_career] > matches[a_col_career]) & (matches['winner'] == 'home')) 
                          + sum((matches[h_col_career] < matches[a_col_career]) & (matches['winner'] == 'away'))) \
                          / total_games

            col_corr_season = (sum((matches[h_col_season] > matches[a_col_season]) & (matches['winner'] == 'home'))
                          + sum((matches[h_col_season] < matches[a_col_season]) & (matches['winner'] == 'away'))) \
                          / total_games

            corr.append(('career_' + col, col_corr_career))
            corr.append(('season_' + col, col_corr_season))
    
    return corr

In [24]:
# Create a dictionary so we can see, by season, how predictive each feature is

from collections import defaultdict

metric_dict = defaultdict(lambda: 0)

for metric in [tup[0] for tup in corr]:
    metric_dict[metric] = defaultdict(lambda: 0)

for idx, row in matches.iterrows():
    print(idx/len(matches))
    
    for metric in metric_dict:
        
        try:
            if ((row['h_' + metric] > row['a_' + metric]) & (row['winner'] == 'home')) | \
                ((row['h_' + metric] < row['a_' + metric]) & (row['winner'] == 'away')):
                metric_dict[metric][row['season']] += 1
        except KeyError:
            pass

for metric in metric_dict:
    for season, value in metric_dict[metric].items():
        try:
            metric_dict[metric][season] = value/sum((matches['season']==season) &
                                                    (matches['h_' + metric] != matches['a_' + metric]))
        except KeyError:
            pass

0.0
6.69702652022502e-05
0.0001339405304045004
0.0002009107956067506
0.0002678810608090008
0.000334851326011251
0.0004018215912135012
0.0004687918564157514
0.0005357621216180017
0.0006027323868202519
0.000669702652022502
0.0007366729172247522
0.0008036431824270024
0.0008706134476292526
0.0009375837128315028
0.001004553978033753
0.0010715242432360033
0.0011384945084382534
0.0012054647736405037
0.0012724350388427538
0.001339405304045004
0.0014063755692472542
0.0014733458344495043
0.0015403160996517546
0.0016072863648540047
0.001674256630056255
0.0017412268952585051
0.0018081971604607555
0.0018751674256630056
0.0019421376908652559
0.002009107956067506
0.0020760782212697563
0.0021430484864720066
0.0022100187516742565
0.002276989016876507
0.002343959282078757
0.0024109295472810074
0.0024778998124832573
0.0025448700776855076
0.002611840342887758
0.002678810608090008
0.002745780873292258
0.0028127511384945084
0.0028797214036967588
0.0029466916688990086
0.003013661934101259
0.00308063219930350

0.02638628448968658
0.02645325475488883
0.02652022502009108
0.026587195285293328
0.02665416555049558
0.02672113581569783
0.02678810608090008
0.02685507634610233
0.02692204661130458
0.02698901687650683
0.027055987141709082
0.02712295740691133
0.027189927672113583
0.02725689793731583
0.027323868202518083
0.027390838467720332
0.027457808732922584
0.027524778998124833
0.02759174926332708
0.027658719528529333
0.027725689793731582
0.027792660058933834
0.027859630324136082
0.027926600589338334
0.027993570854540583
0.028060541119742835
0.028127511384945084
0.028194481650147336
0.028261451915349584
0.028328422180551836
0.028395392445754085
0.028462362710956337
0.028529332976158586
0.028596303241360834
0.028663273506563086
0.028730243771765335
0.028797214036967587
0.028864184302169835
0.028931154567372087
0.028998124832574336
0.029065095097776588
0.029132065362978837
0.02919903562818109
0.029266005893383337
0.02933297615858559
0.029399946423787838
0.02946691668899009
0.02953388695419234
0.029600

0.054111974283418164
0.05417894454862041
0.05424591481382266
0.05431288507902491
0.054379855344227165
0.054446825609429414
0.05451379587463166
0.05458076613983391
0.05464773640503617
0.054714706670238415
0.054781676935440664
0.05484864720064291
0.05491561746584517
0.054982587731047416
0.055049557996249665
0.055116528261451914
0.05518349852665416
0.05525046879185642
0.055317439057058666
0.055384409322260915
0.055451379587463163
0.05551834985266542
0.05558532011786767
0.055652290383069916
0.055719260648272165
0.05578623091347442
0.05585320117867667
0.05592017144387892
0.055987141709081166
0.05605411197428342
0.05612108223948567
0.05618805250468792
0.05625502276989017
0.056321993035092416
0.05638896330029467
0.05645593356549692
0.05652290383069917
0.05658987409590142
0.05665684436110367
0.05672381462630592
0.05679078489150817
0.05685775515671042
0.056924725421912674
0.05699169568711492
0.05705866595231717
0.05712563621751942
0.05719260648272167
0.057259576747923924
0.05732654701312617
0.0

0.08210554513795874
0.08217251540316099
0.08223948566836324
0.0823064559335655
0.08237342619876775
0.08244039646397
0.08250736672917225
0.0825743369943745
0.08264130725957675
0.082708277524779
0.08277524778998124
0.08284221805518349
0.08290918832038575
0.082976158585588
0.08304312885079025
0.0831100991159925
0.08317706938119475
0.083244039646397
0.08331100991159925
0.0833779801768015
0.08344495044200376
0.083511920707206
0.08357889097240825
0.0836458612376105
0.08371283150281275
0.083779801768015
0.08384677203321725
0.0839137422984195
0.08398071256362175
0.08404768282882401
0.08411465309402626
0.0841816233592285
0.08424859362443075
0.084315563889633
0.08438253415483525
0.0844495044200375
0.08451647468523975
0.08458344495044201
0.08465041521564426
0.08471738548084651
0.08478435574604876
0.084851326011251
0.08491829627645325
0.0849852665416555
0.08505223680685775
0.08511920707206
0.08518617733726226
0.08525314760246451
0.08532011786766676
0.08538708813286901
0.08545405839807126
0.0855210

0.11137155103134208
0.11143852129654433
0.11150549156174658
0.11157246182694884
0.11163943209215109
0.11170640235735334
0.11177337262255559
0.11184034288775783
0.11190731315296008
0.11197428341816233
0.11204125368336458
0.11210822394856684
0.11217519421376909
0.11224216447897134
0.11230913474417359
0.11237610500937584
0.11244307527457809
0.11251004553978033
0.11257701580498258
0.11264398607018483
0.1127109563353871
0.11277792660058934
0.11284489686579159
0.11291186713099384
0.11297883739619609
0.11304580766139834
0.11311277792660059
0.11317974819180283
0.1132467184570051
0.11331368872220735
0.1133806589874096
0.11344762925261184
0.11351459951781409
0.11358156978301634
0.11364854004821859
0.11371551031342084
0.11378248057862309
0.11384945084382535
0.1139164211090276
0.11398339137422984
0.1140503616394321
0.11411733190463434
0.11418430216983659
0.11425127243503884
0.11431824270024109
0.11438521296544334
0.1144521832306456
0.11451915349584785
0.1145861237610501
0.11465309402625234
0.11472

...

0.8643852129654434
0.8644521832306455
0.8645191534958478
0.86458612376105
0.8646530940262523
0.8647200642914546
0.8647870345566568
0.8648540048218591
0.8649209750870613
0.8649879453522636
0.8650549156174658
0.8651218858826681
0.8651888561478703
0.8652558264130726
0.8653227966782748
0.8653897669434771
0.8654567372086793
0.8655237074738816
0.8655906777390838
0.8656576480042861
0.8657246182694883
0.8657915885346906
0.8658585587998928
0.8659255290650951
0.8659924993302973
0.8660594695954996
0.8661264398607018
0.8661934101259041
0.8662603803911063
0.8663273506563086
0.8663943209215108
0.8664612911867131
0.8665282614519153
0.8665952317171176
0.8666622019823198
0.8667291722475221
0.8667961425127243
0.8668631127779266
0.8669300830431288
0.8669970533083311
0.8670640235735334
0.8671309938387356
0.8671979641039379
0.8672649343691401
0.8673319046343424
0.8673988748995446
0.8674658451647469
0.8675328154299491
0.8675997856951514
0.8676667559603536
0.8677337262255559
0.8678006964907581
0.867867666755

0.8933163675328154
0.8933833377980177
0.8934503080632199
0.8935172783284222
0.8935842485936244
0.8936512188588267
0.8937181891240289
0.8937851593892312
0.8938521296544334
0.8939190999196357
0.8939860701848379
0.8940530404500402
0.8941200107152424
0.8941869809804447
0.8942539512456469
0.8943209215108492
0.8943878917760514
0.8944548620412537
0.894521832306456
0.8945888025716582
0.8946557728368605
0.8947227431020627
0.894789713367265
0.8948566836324672
0.8949236538976695
0.8949906241628717
0.895057594428074
0.8951245646932762
0.8951915349584785
0.8952585052236807
0.895325475488883
0.8953924457540852
0.8954594160192875
0.8955263862844897
0.895593356549692
0.8956603268148942
0.8957272970800965
0.8957942673452987
0.895861237610501
0.8959282078757032
0.8959951781409055
0.8960621484061077
0.89612911867131
0.8961960889365121
0.8962630592017145
0.8963300294669166
0.896396999732119
0.8964639699973211
0.8965309402625234
0.8965979105277256
0.8966648807929279
0.8967318510581301
0.8967988213233324
0.

0.9252611840342888
0.925328154299491
0.9253951245646933
0.9254620948298955
0.9255290650950978
0.9255960353603
0.9256630056255023
0.9257299758907045
0.9257969461559068
0.925863916421109
0.9259308866863113
0.9259978569515135
0.9260648272167158
0.926131797481918
0.9261987677471203
0.9262657380123225
0.9263327082775248
0.926399678542727
0.9264666488079293
0.9265336190731315
0.9266005893383338
0.9266675596035361
0.9267345298687383
0.9268015001339406
0.9268684703991428
0.9269354406643451
0.9270024109295473
0.9270693811947496
0.9271363514599518
0.9272033217251541
0.9272702919903563
0.9273372622555586
0.9274042325207608
0.9274712027859631
0.9275381730511653
0.9276051433163676
0.9276721135815698
0.9277390838467721
0.9278060541119743
0.9278730243771766
0.9279399946423788
0.9280069649075811
0.9280739351727832
0.9281409054379856
0.9282078757031877
0.92827484596839
0.9283418162335922
0.9284087864987945
0.9284757567639967
0.928542727029199
0.9286096972944012
0.9286766675596035
0.9287436378248057
0.9

0.9551299223144923
0.9551968925796946
0.9552638628448968
0.9553308331100991
0.9553978033753013
0.9554647736405036
0.9555317439057058
0.9555987141709081
0.9556656844361103
0.9557326547013126
0.9557996249665148
0.9558665952317171
0.9559335654969193
0.9560005357621216
0.9560675060273238
0.9561344762925261
0.9562014465577283
0.9562684168229306
0.9563353870881328
0.9564023573533351
0.9564693276185374
0.9565362978837396
0.9566032681489419
0.9566702384141441
0.9567372086793464
0.9568041789445486
0.9568711492097509
0.9569381194749531
0.9570050897401554
0.9570720600053576
0.9571390302705599
0.9572060005357621
0.9572729708009644
0.9573399410661666
0.9574069113313689
0.9574738815965711
0.9575408518617734
0.9576078221269756
0.9576747923921779
0.9577417626573801
0.9578087329225824
0.9578757031877846
0.9579426734529869
0.9580096437181891
0.9580766139833914
0.9581435842485936
0.9582105545137959
0.9582775247789981
0.9583444950442004
0.9584114653094026
0.9584784355746049
0.9585454058398071
0.9586123761

0.9868068577551568
0.986873828020359
0.9869407982855612
0.9870077685507634
0.9870747388159657
0.9871417090811679
0.9872086793463702
0.9872756496115724
0.9873426198767747
0.9874095901419769
0.9874765604071792
0.9875435306723814
0.9876105009375837
0.9876774712027859
0.9877444414679882
0.9878114117331904
0.9878783819983927
0.9879453522635949
0.9880123225287972
0.9880792927939994
0.9881462630592017
0.9882132333244039
0.9882802035896062
0.9883471738548084
0.9884141441200107
0.9884811143852129
0.9885480846504152
0.9886150549156175
0.9886820251808197
0.988748995446022
0.9888159657112242
0.9888829359764265
0.9889499062416287
0.989016876506831
0.9890838467720332
0.9891508170372355
0.9892177873024377
0.98928475756764
0.9893517278328422
0.9894186980980445
0.9894856683632467
0.989552638628449
0.9896196088936512
0.9896865791588535
0.9897535494240557
0.989820519689258
0.9898874899544602
0.9899544602196625
0.9900214304848647
0.990088400750067
0.9901553710152692
0.9902223412804715
0.9902893115456737
0

In [35]:
# Checking how often the favourite wins

relevant_games = matches[(matches['season'] >= 2010) & (matches['homeodds'] != matches['awayodds'])]
total_games = len(relevant_games)
odds_corr = (sum((matches['homeodds'] < matches['awayodds']) & (matches['winner'] == 'home')) 
              + sum((matches['homeodds'] > matches['awayodds']) & (matches['winner'] == 'away'))) \
              / total_games

In [65]:
# Check how often the favourite wins for each season
from collections import defaultdict

odds_dict = defaultdict(lambda: 0)

for idx, row in matches.iterrows():
    print(idx/len(matches))

    if ((row['homeodds'] < row['awayodds']) & (row['winner'] == 'home')) | \
        ((row['homeodds'] > row['awayodds']) & (row['winner'] == 'away')):
        odds_dict[row['season']] += 1

for season, value in odds_dict.items():
    odds_dict[season] = value/sum(matches['season']==season)

0.0
6.69702652022502e-05
0.0001339405304045004
0.0002009107956067506
0.0002678810608090008
0.000334851326011251
0.0004018215912135012
0.0004687918564157514
0.0005357621216180017
0.0006027323868202519
0.000669702652022502
0.0007366729172247522
0.0008036431824270024
0.0008706134476292526
0.0009375837128315028
0.001004553978033753
0.0010715242432360033
0.0011384945084382534
0.0012054647736405037
0.0012724350388427538
0.001339405304045004
0.0014063755692472542
0.0014733458344495043
0.0015403160996517546
0.0016072863648540047
0.001674256630056255
0.0017412268952585051
0.0018081971604607555
0.0018751674256630056
0.0019421376908652559
0.002009107956067506
0.0020760782212697563
0.0021430484864720066
0.0022100187516742565
0.002276989016876507
0.002343959282078757
0.0024109295472810074
0.0024778998124832573
0.0025448700776855076
0.002611840342887758
0.002678810608090008
0.002745780873292258
0.0028127511384945084
0.0028797214036967588
0.0029466916688990086
0.003013661934101259
0.00308063219930350

0.04580766139833914
0.04587463166354139
0.045941601928743636
0.046008572193945885
0.04607554245914814
0.04614251272435039
0.04620948298955264
0.046276453254754886
0.04634342351995714
0.04641039378515939
0.04647736405036164
0.04654433431556389
0.04661130458076614
0.04667827484596839
0.04674524511117064
0.04681221537637289
0.046879185641575144
0.04694615590677739
0.04701312617197964
0.04708009643718189
0.04714706670238414
0.047214036967586394
0.04728100723278864
0.04734797749799089
0.04741494776319314
0.047481918028395395
0.047548888293597644
0.04761585855879989
0.04768282882400214
0.047749799089204396
0.047816769354406645
0.047883739619608894
0.04795070988481114
0.04801768015001339
0.048084650415215646
0.048151620680417895
0.04821859094562014
0.04828556121082239
0.04835253147602465
0.048419501741226896
0.048486472006429145
0.04855344227163139
0.04862041253683365
0.0486873828020359
0.048754353067238146
0.048821323332440394
0.04888829359764265
0.0489552638628449
0.04902223412804715
0.0490

0.07614519153495848
0.07621216180016073
0.07627913206536298
0.07634610233056523
0.07641307259576748
0.07648004286096972
0.07654701312617197
0.07661398339137424
0.07668095365657648
0.07674792392177873
0.07681489418698098
0.07688186445218323
0.07694883471738548
0.07701580498258773
0.07708277524778998
0.07714974551299224
0.07721671577819449
0.07728368604339673
0.07735065630859898
0.07741762657380123
0.07748459683900348
0.07755156710420573
0.07761853736940798
0.07768550763461023
0.07775247789981249
0.07781944816501474
0.07788641843021699
0.07795338869541923
0.07802035896062148
0.07808732922582373
0.07815429949102598
0.07822126975622823
0.07828824002143049
0.07835521028663274
0.07842218055183499
0.07848915081703724
0.07855612108223949
0.07862309134744173
0.07869006161264398
0.07875703187784623
0.07882400214304848
0.07889097240825074
0.07895794267345299
0.07902491293865524
0.07909188320385749
0.07915885346905974
0.07922582373426199
0.07929279399946423
0.07935976426466648
0.07942673452986874


0.1328690061612644
0.13293597642646665
0.1330029466916689
0.13306991695687115
0.1331368872220734
0.13320385748727565
0.1332708277524779
0.13333779801768014
0.1334047682828824
0.13347173854808464
0.1335387088132869
0.13360567907848916
0.1336726493436914
0.13373961960889366
0.1338065898740959
0.13387356013929816
0.1339405304045004
0.13400750066970266
0.1340744709349049
0.13414144120010715
0.1342084114653094
0.13427538173051165
0.1343423519957139
0.13440932226091615
0.1344762925261184
0.13454326279132064
0.1346102330565229
0.13467720332172514
0.1347441735869274
0.13481114385212967
0.13487811411733192
0.13494508438253416
0.1350120546477364
0.13507902491293866
0.1351459951781409
0.13521296544334316
0.1352799357085454
0.13534690597374766
0.1354138762389499
0.13548084650415215
0.1355478167693544
0.13561478703455665
0.1356817572997589
0.13574872756496115
0.1358156978301634
0.13588266809536564
0.13594963836056792
0.13601660862577017
0.13608357889097242
0.13615054915617467
0.13621751942137691
0.

0.19026252343959282
0.19032949370479507
0.19039646396999732
0.19046343423519957
0.19053040450040182
0.19059737476560407
0.19066434503080631
0.19073131529600856
0.1907982855612108
0.19086525582641306
0.1909322260916153
0.19099919635681759
0.19106616662201983
0.19113313688722208
0.19120010715242433
0.19126707741762658
0.19133404768282883
0.19140101794803108
0.19146798821323333
0.19153495847843557
0.19160192874363782
0.19166889900884007
0.19173586927404232
0.19180283953924457
0.19186980980444682
0.19193678006964907
0.19200375033485131
0.19207072060005356
0.19213769086525584
0.1922046611304581
0.19227163139566034
0.19233860166086258
0.19240557192606483
0.19247254219126708
0.19253951245646933
0.19260648272167158
0.19267345298687383
0.19274042325207608
0.19280739351727832
0.19287436378248057
0.19294133404768282
0.19300830431288507
0.19307527457808732
0.19314224484328957
0.19320921510849182
0.1932761853736941
0.19334315563889634
0.1934101259040986
0.19347709616930084
0.1935440664345031
0.1936

...

0.7897133672649344
0.7897803375301367
0.7898473077953388
0.7899142780605412
0.7899812483257433
0.7900482185909457
0.7901151888561478
0.7901821591213501
0.7902491293865523
0.7903160996517546
0.7903830699169568
0.7904500401821591
0.7905170104473613
0.7905839807125636
0.7906509509777658
0.7907179212429681
0.7907848915081703
0.7908518617733726
0.7909188320385748
0.7909858023037771
0.7910527725689793
0.7911197428341816
0.7911867130993838
0.7912536833645861
0.7913206536297884
0.7913876238949906
0.7914545941601929
0.7915215644253951
0.7915885346905974
0.7916555049557996
0.7917224752210019
0.7917894454862041
0.7918564157514064
0.7919233860166086
0.7919903562818109
0.7920573265470131
0.7921242968122154
0.7921912670774176
0.7922582373426199
0.7923252076078221
0.7923921778730244
0.7924591481382266
0.7925261184034289
0.7925930886686311
0.7926600589338334
0.7927270291990356
0.7927939994642379
0.7928609697294401
0.7929279399946424
0.7929949102598446
0.7930618805250469
0.7931288507902491
0.7931958210

0.8414813822662738
0.841548352531476
0.8416153227966783
0.8416822930618805
0.8417492633270828
0.8418162335922851
0.8418832038574873
0.8419501741226896
0.8420171443878918
0.8420841146530941
0.8421510849182963
0.8422180551834986
0.8422850254487008
0.8423519957139031
0.8424189659791053
0.8424859362443076
0.8425529065095098
0.8426198767747121
0.8426868470399143
0.8427538173051166
0.8428207875703188
0.8428877578355211
0.8429547281007232
0.8430216983659256
0.8430886686311277
0.84315563889633
0.8432226091615322
0.8432895794267345
0.8433565496919367
0.843423519957139
0.8434904902223412
0.8435574604875435
0.8436244307527457
0.843691401017948
0.8437583712831502
0.8438253415483525
0.8438923118135547
0.843959282078757
0.8440262523439592
0.8440932226091615
0.8441601928743638
0.844227163139566
0.8442941334047683
0.8443611036699705
0.8444280739351728
0.844495044200375
0.8445620144655773
0.8446289847307795
0.8446959549959818
0.844762925261184
0.8448298955263863
0.8448968657915885
0.8449638360567908
0.

0.8876238949906242
0.8876908652558264
0.8877578355210287
0.8878248057862309
0.8878917760514332
0.8879587463166354
0.8880257165818377
0.8880926868470399
0.8881596571122422
0.8882266273774444
0.8882935976426467
0.8883605679078489
0.8884275381730512
0.8884945084382534
0.8885614787034557
0.8886284489686579
0.8886954192338602
0.8887623894990624
0.8888293597642647
0.8888963300294669
0.8889633002946692
0.8890302705598714
0.8890972408250737
0.8891642110902759
0.8892311813554782
0.8892981516206804
0.8893651218858827
0.8894320921510849
0.8894990624162872
0.8895660326814894
0.8896330029466917
0.8896999732118939
0.8897669434770962
0.8898339137422984
0.8899008840075007
0.889967854272703
0.8900348245379052
0.8901017948031075
0.8901687650683097
0.890235735333512
0.8903027055987142
0.8903696758639165
0.8904366461291187
0.890503616394321
0.8905705866595232
0.8906375569247255
0.8907045271899277
0.89077149745513
0.8908384677203322
0.8909054379855345
0.8909724082507366
0.891039378515939
0.8911063487811411

0.9574069113313689
0.9574738815965711
0.9575408518617734
0.9576078221269756
0.9576747923921779
0.9577417626573801
0.9578087329225824
0.9578757031877846
0.9579426734529869
0.9580096437181891
0.9580766139833914
0.9581435842485936
0.9582105545137959
0.9582775247789981
0.9583444950442004
0.9584114653094026
0.9584784355746049
0.9585454058398071
0.9586123761050094
0.9586793463702116
0.9587463166354139
0.9588132869006162
0.9588802571658184
0.9589472274310207
0.9590141976962229
0.9590811679614252
0.9591481382266274
0.9592151084918297
0.9592820787570319
0.9593490490222342
0.9594160192874364
0.9594829895526387
0.9595499598178409
0.9596169300830432
0.9596839003482454
0.9597508706134477
0.9598178408786499
0.9598848111438522
0.9599517814090543
0.9600187516742567
0.9600857219394588
0.9601526922046612
0.9602196624698633
0.9602866327350656
0.9603536030002678
0.9604205732654701
0.9604875435306723
0.9605545137958746
0.9606214840610768
0.9606884543262791
0.9607554245914813
0.9608223948566836
0.9608893651

0.9904232520760782
0.9904902223412805
0.9905571926064827
0.990624162871685
0.9906911331368872
0.9907581034020895
0.9908250736672917
0.990892043932494
0.9909590141976963
0.9910259844628985
0.9910929547281008
0.991159924993303
0.9912268952585053
0.9912938655237075
0.9913608357889098
0.991427806054112
0.9914947763193143
0.9915617465845165
0.9916287168497188
0.991695687114921
0.9917626573801233
0.9918296276453255
0.9918965979105278
0.99196356817573
0.9920305384409323
0.9920975087061344
0.9921644789713367
0.9922314492365389
0.9922984195017412
0.9923653897669434
0.9924323600321457
0.9924993302973479
0.9925663005625502
0.9926332708277524
0.9927002410929547
0.9927672113581569
0.9928341816233592
0.9929011518885614
0.9929681221537637
0.9930350924189659
0.9931020626841682
0.9931690329493705
0.9932360032145727
0.993302973479775
0.9933699437449772
0.9934369140101795
0.9935038842753817
0.993570854540584
0.9936378248057862
0.9937047950709885
0.9937717653361907
0.993838735601393
0.9939057058665952
0.9

Read in current matches

In [69]:
df_path = '/Users/t_raver9/Desktop/projects/aflengine/bench/matches.csv'
df = pd.read_csv(df_path)

In [85]:
df[(df['round'] == '23') & (df['season'] == 2018)]

,round,venue,date,day,time,crowd,hteam,hteam_q1,hteam_q2,hteam_q3,hteam_q4,ateam,ateam_q1,ateam_q2,ateam_q3,ateam_q4,umpire1,umpire2,umpire3,umpire1games,umpire2games,umpire3games,hteam_et,ateam_et,season,matchid,homeodds,homeline,awayodds,awayline,hscore,ascore
8135,23,Adelaide Oval,24-Aug-2018,Fri,7:20,39168,Port Adelaide,1.1.7,5.6.36,9.7.61,13.11.89,Essendon,6.4.40,9.7.61,15.8.98,17.9.111,Jacob Mollison,Rob Findlay,Dean Margetts,215,210.0,322.0,NaN,NaN,2018,201823ESSPOR,1.64,-10.5,2.30,10.5,89,111
8667,23,Docklands,25-Aug-2018,Sat,7:25,17000,Carlton,1.4.10,5.5.35,7.8.50,8.13.61,Adelaide,5.4.34,13.6.84,20.8.128,26.9.165,Brendan Hosking,Andrew Stephens,Leigh Fisher,131,86.0,114.0,NaN,NaN,2018,201823ADECAR,15.00,55.5,1.02,-55.5,61,165
8668,23,Perth Stadium,25-Aug-2018,Sat,2:35,41320,Fremantle,5.2.32,6.6.42,8.8.56,9.13.67,Collingwood,3.2.20,6.3.39,8.8.56,11.10.76,Sam Hay,Mathew Nicholls,Rob OGorman,179,331.0,86.0,NaN,NaN,2018,201823COLFRE,6.00,35.5,1.14,-35.5,67,76
8669,23,Kardinia Park,25-Aug-2018,Sat,1:45,28004,Geelong,4.6.30,12.8.80,17.8.110,22.10.142,Gold Coast,2.1.13,3.2.20,4.8.32,5.10.40,Nathan Williamson,Hayden Gavine,Chris Donlon,44,29.0,290.0,NaN,NaN,2018,201823GCSGEE,1.01,-79.5,21.00,79.5,142,40
8670,23,M.C.G.,25-Aug-2018,Sat,2:10,56998,Richmond,5.1.31,7.7.49,12.8.80,15.8.98,Footscray,6.0.36,8.3.51,11.6.72,14.11.95,Eleni Glouftsis,Troy Pannell,Justin Schmitt,16,219.0,363.0,NaN,NaN,2018,201823FOTRCH,1.08,-45.5,8.50,45.5,98,95
8671,23,S.C.G.,25-Aug-2018,Sat,7:25,39660,Sydney,2.3.15,6.7.43,8.8.56,10.14.74,Hawthorn,2.4.16,3.6.24,7.8.50,12.11.83,David Harris,Simon Meredith,Ray Chamberlain,114,339.0,284.0,NaN,NaN,2018,201823HAWSYD,2.40,10.5,1.57,-10.5,74,83
9184,23,M.C.G.,26-Aug-2018,Sun,3:20,37285,Melbourne,4.2.26,7.6.48,12.9.81,15.12.102,Greater Western Sydney,4.1.25,5.5.35,5.8.38,8.9.57,Matt Stevic,Brett Rosebury,Jeff Dalgleish,352,395.0,189.0,NaN,NaN,2018,201823GWSMEL,1.45,-17.5,2.85,17.5,102,57
9185,23,Docklands,26-Aug-2018,Sun,4:40,19866,St Kilda,3.3.21,6.4.40,11.6.72,14.10.94,North Melbourne,6.7.43,11.9.75,14.11.95,17.15.117,Shane McInerney,Alex Whetton,Curtis Deboy,484,11.0,74.0,NaN,NaN,2018,201823NORSTK,3.35,20.5,1.35,-20.5,94,117
9186,23,Gabba,26-Aug-2018,Sun,1:10,16367,Brisbane Lions,4.0.24,4.2.26,7.6.48,11.6.72,West Coast,5.4.34,9.7.61,12.12.84,14.14.98,Craig Fleer,Shaun Ryan,Leigh Haussen,99,301.0,33.0,NaN,NaN,2018,201823BRSWEG,2.20,6.5,1.70,-6.5,72,98


In [86]:
df[(df['round'] == '22') & (df['season'] == 2018)]

,round,venue,date,day,time,crowd,hteam,hteam_q1,hteam_q2,hteam_q3,hteam_q4,ateam,ateam_q1,ateam_q2,ateam_q3,ateam_q4,umpire1,umpire2,umpire3,umpire1games,umpire2games,umpire3games,hteam_et,ateam_et,season,matchid,homeodds,homeline,awayodds,awayline,hscore,ascore
4110,22,M.C.G.,17-Aug-2018,Fri,7:50,76424,Richmond,4.0.24,5.6.36,10.8.68,12.9.81,Essendon,2.3.15,4.4.28,7.5.47,11.7.73,Shane McInerney,David Harris,Ray Chamberlain,483,113.0,283.0,NaN,NaN,2018,201822ESSRCH,1.45,-14.5,2.85,14.5,81,73
4602,22,Kardinia Park,18-Aug-2018,Sat,2:10,24507,Geelong,1.5.11,11.6.72,17.9.111,24.14.158,Fremantle,3.2.20,3.2.20,3.5.23,3.7.25,Sam Hay,Andrew Stephens,Brett Rosebury,178,85.0,394.0,NaN,NaN,2018,201822FREGEE,1.05,-49.5,11.00,49.5,158,25
4603,22,Docklands,18-Aug-2018,Sat,7:25,24795,St Kilda,1.3.9,6.6.42,10.9.69,11.10.76,Hawthorn,4.4.28,7.5.47,12.6.78,12.8.80,Eleni Glouftsis,Nathan Williamson,Dean Margetts,15,43.0,321.0,NaN,NaN,2018,201822HAWSTK,8.40,39.5,1.06,-39.5,76,80
4604,22,Carrara,18-Aug-2018,Sat,7:25,11907,Gold Coast,1.2.8,6.4.40,8.6.54,11.8.74,Brisbane Lions,3.5.23,6.10.46,7.15.57,10.18.78,Simon Meredith,Alex Whetton,Leigh Haussen,338,10.0,32.0,NaN,NaN,2018,201822BRSGCS,6.00,35.5,1.13,-35.5,74,78
4605,22,Sydney Showground,18-Aug-2018,Sat,4:35,21433,Greater Western Sydney,3.3.21,6.6.42,7.9.51,8.12.60,Sydney,1.2.8,4.3.27,5.9.39,11.14.80,Jacob Mollison,Troy Pannell,Curtis Deboy,214,218.0,73.0,NaN,NaN,2018,201822GWSSYD,1.77,-4.5,2.10,4.5,60,80
4606,22,M.C.G.,18-Aug-2018,Sat,1:45,46286,Collingwood,4.2.26,9.7.61,10.11.71,17.13.115,Port Adelaide,4.1.25,7.2.44,9.4.58,10.4.64,Craig Fleer,Mathew Nicholls,Jeff Dalgleish,98,330.0,188.0,NaN,NaN,2018,201822COLPOR,1.45,-15.5,2.85,15.5,115,64
5116,22,Docklands,19-Aug-2018,Sun,1:10,24143,Carlton,2.2.14,4.4.28,5.6.36,7.7.49,Footscray,2.1.13,4.2.26,6.4.40,10.6.66,Shaun Ryan,Hayden Gavine,Nick Brown,300,28.0,36.0,NaN,NaN,2018,201822CARFOT,4.50,28.5,1.22,-28.5,49,66
5117,22,Perth Stadium,19-Aug-2018,Sun,1:20,55824,West Coast,3.0.18,5.4.34,11.6.72,14.7.91,Melbourne,4.4.28,7.6.48,12.8.80,16.12.108,Justin Schmitt,Brendan Hosking,Matt Stevic,362,130.0,351.0,NaN,NaN,2018,201822MELWEG,1.71,-6.5,2.20,6.5,91,108
5118,22,Adelaide Oval,19-Aug-2018,Sun,4:10,41444,Adelaide,3.5.23,7.9.51,10.13.73,12.14.86,North Melbourne,3.2.20,3.5.23,6.7.43,11.11.77,Rob Findlay,Rob OGorman,Chris Donlon,209,85.0,289.0,NaN,NaN,2018,201822ADENOR,1.40,-18.5,3.05,18.5,86,77


In [84]:
pd.set_option('display.max_columns',500)

In [90]:
from typing import (
    Type,
    List
)

class Team:
    """
    Holds team-level data for a particular match
    """
    def __init__(self, generic_team_columns: List, home_or_away: str):
        self.home_or_away = home_or_away
        for column in generic_team_columns:
            setattr(self, column, None)

    def add_data(self, data: pd.DataFrame):
        if self.home_or_away == 'home':
            for home_col, generic_col in home_cols_mapped.items():
                self.__dict__[generic_col] = data[home_col]
        if self.home_or_away == 'away':
            for away_col, generic_col in away_cols_mapped.items():
                self.__dict__[generic_col] = data[away_col]

class Match:
    """
    Holds data about a match, as well as an object for each team
    """
    def __init__(self, match_columns: List):
        self.teams = {
            'home': None,
            'away': None
        }
        for column in match_columns:
            setattr(self, column, None)

    def add_data(self, data: pd.DataFrame):
        for column in self.__dict__.keys():
            try:
                self.__dict__[column] = data[column]
            except KeyError:
                continue
    
    def add_home_team(self, team: Type[Team]):
        self.teams['home'] = team
    
    def add_away_team(self, team: Type[Team]):
        self.teams['away'] = team

class Round:
    """
    Contains match and ladder data for each round
    """
    def __init__(self, round_num: int):
        self.round_num = round_num
        self.matches = []
        self.bye_teams = []
        self.ladder = None

    def add_match(self, match: Type[Match]):
        self.matches.append(match)

    def add_ladder(self, ladder):
        self.ladder = ladder

class Season:
    """
    Contains the rounds for a season, and which teams competed
    """
    def __init__(self, year: int, teams: List):
        self.year = year
        self.teams = teams
        self.rounds = {}
        
    def add_round(self, round_obj: Type[Round]):
        self.rounds[round_obj.round_num] = round_obj

class History:
    """
    Holds all season objects
    """
    def __init__(self):
        self.seasons = {}
        
    def add_season(self, season: Type[Season]):
        self.seasons[season.year] = season

class TeamLadder:
    def __init__(self, team: str):
        self.team = team
        for column, init_val in ladder_columns:
            setattr(self, column, init_val)

    def add_prev_round_team_ladder(self, prev_round_team_ladder):
        for col,val in prev_round_team_ladder.items():
            self.__dict__[col] = val

    def update_home_team(self, match: Type[Match]):
        self.played += 1
        if match.hscore > match.ascore:
            self.wins += 1
            self.prem_points += 4
        elif match.hscore == match.ascore:
            self.draws += 1
            self.prem_points += 2
        else:
            self.losses += 1    
        self.points_for += match.hscore
        self.points_against += match.ascore
        self.percentage = 100 * (self.points_for / self.points_against)

    def update_away_team(self, match: Type[Match]):
        self.played += 1
        if match.hscore < match.ascore:
            self.wins += 1
            self.prem_points += 4
        elif match.hscore == match.ascore:
            self.draws += 1
            self.prem_points += 2
        else:
            self.losses += 1    
        self.points_for += match.ascore
        self.points_against += match.hscore
        self.percentage = 100 * (self.points_for / self.points_against)

    def update_ladder(self, match: Type[Match]):
        """
        Update the ladder for the team based on the outcome of the game. There
        will be two possibilites - the team can be the home or the away team
        in the provided match.
        """
        if self.team == match.teams['home']:
            self.update_home_team(match)
        else:
            self.update_away_team(match)

class Ladder:
    """
    Each round object holds the ladder details for that round for each team
    """
    def __init__(self, teams_in_season: List):
        self.teams_in_season = teams_in_season
        self.team_ladders = {}

    def add_team_ladder(self, team_ladder: Type[TeamLadder]):
        self.team_ladders[team_ladder.team.team] = team_ladder
        


In [112]:
# read in pickle object
history_ladder_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/ladder_data/data/history_object.pk1'

In [113]:
import pickle
with open(history_ladder_path, 'rb') as f:
        history_ladder_path = pickle.load(f)

In [114]:
history_ladder_path.__dict__

{'seasons': {1897: <src.match_data.generate_match_objects.Season at 0x122ee5610>,
  1898: <src.match_data.generate_match_objects.Season at 0x123a39490>,
  1899: <src.match_data.generate_match_objects.Season at 0x124161910>,
  1900: <src.match_data.generate_match_objects.Season at 0x1244958d0>,
  1901: <src.match_data.generate_match_objects.Season at 0x124976650>,
  1902: <src.match_data.generate_match_objects.Season at 0x12509bd90>,
  1903: <src.match_data.generate_match_objects.Season at 0x1255e1590>,
  1904: <src.match_data.generate_match_objects.Season at 0x125b42150>,
  1905: <src.match_data.generate_match_objects.Season at 0x1264d48d0>,
  1906: <src.match_data.generate_match_objects.Season at 0x1285f0910>,
  1907: <src.match_data.generate_match_objects.Season at 0x128b58ed0>,
  1908: <src.match_data.generate_match_objects.Season at 0x128f75690>,
  1909: <src.match_data.generate_match_objects.Season at 0x1296fe890>,
  1910: <src.match_data.generate_match_objects.Season at 0x12ae9c6

In [115]:
history_ladder_path.__dict__['seasons'][2018].__dict__

{'year': 2018,
 'teams': ['Adelaide',
  'Brisbane Lions',
  'Carlton',
  'Collingwood',
  'Essendon',
  'Footscray',
  'Fremantle',
  'Geelong',
  'Gold Coast',
  'Greater Western Sydney',
  'Hawthorn',
  'Melbourne',
  'North Melbourne',
  'Port Adelaide',
  'Richmond',
  'St Kilda',
  'Sydney',
  'West Coast'],
 'rounds': {1: <src.match_data.generate_match_objects.Round at 0x137cf6ed0>,
  2: <src.match_data.generate_match_objects.Round at 0x137d05cd0>,
  3: <src.match_data.generate_match_objects.Round at 0x137d0ff90>,
  4: <src.match_data.generate_match_objects.Round at 0x137d1e450>,
  5: <src.match_data.generate_match_objects.Round at 0x137d2a410>,
  6: <src.match_data.generate_match_objects.Round at 0x137d344d0>,
  7: <src.match_data.generate_match_objects.Round at 0x137d40550>,
  8: <src.match_data.generate_match_objects.Round at 0x137d4b550>,
  9: <src.match_data.generate_match_objects.Round at 0x137d57710>,
  10: <src.match_data.generate_match_objects.Round at 0x137d626d0>,
  11

In [116]:
history_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/match_data/data/history_object.pk1'
with open(history_path, 'rb') as f:
        history = pickle.load(f)

In [118]:
history.__dict__['seasons'][2018].__dict__

{'year': 2018,
 'teams': ['Adelaide',
  'Brisbane Lions',
  'Carlton',
  'Collingwood',
  'Essendon',
  'Footscray',
  'Fremantle',
  'Geelong',
  'Gold Coast',
  'Greater Western Sydney',
  'Hawthorn',
  'Melbourne',
  'North Melbourne',
  'Port Adelaide',
  'Richmond',
  'St Kilda',
  'Sydney',
  'West Coast'],
 'rounds': {1: <src.match_data.generate_match_objects.Round at 0x1397b15d0>,
  2: <src.match_data.generate_match_objects.Round at 0x1397b6ed0>,
  3: <src.match_data.generate_match_objects.Round at 0x1397bd850>,
  4: <src.match_data.generate_match_objects.Round at 0x1397c4390>,
  5: <src.match_data.generate_match_objects.Round at 0x1397c99d0>,
  6: <src.match_data.generate_match_objects.Round at 0x1397d0150>,
  7: <src.match_data.generate_match_objects.Round at 0x1397d4850>,
  8: <src.match_data.generate_match_objects.Round at 0x1397d9ed0>,
  9: <src.match_data.generate_match_objects.Round at 0x1397df750>,
  10: <src.match_data.generate_match_objects.Round at 0x1397e4d90>,
  11

In [120]:
players

NameError: name 'players' is not defined